In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

In [4]:
import csv
import os

# Directory to save CSV files
output_dir = '.\data\dataset4'
os.makedirs(output_dir, exist_ok=True)

gesture_action = "WAVING"

# Function to record hand landmarks
def record_hand_landmarks():
    cap = cv2.VideoCapture(0)
    recording = False
    landmarks_seq = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                if recording:
                    # Extract landmarks
                    landmarks = [lm for lm in hand_landmarks.landmark]
                    landmarks_flat = [coord for lm in landmarks for coord in (lm.x, lm.y, lm.z)]
                    landmarks_seq.append(landmarks_flat)

        cv2.imshow('Hand Gesture Recording', frame)

        key = cv2.waitKey(5) & 0xFF
        if key == ord('r'):
            recording = True
            landmarks_seq = []  # Start a new recording
            print("Recording started...")
        elif key == ord('s'):
            recording = False
            print("Recording stopped.")
            if landmarks_seq:
                # Save to CSV
                gesture_name = gesture_action
                current_time = int(time.time())

                output_file = os.path.join(output_dir, f'{gesture_name}_{current_time}.csv')
                with open(output_file, 'w', newline='') as f:
                    writer = csv.writer(f)
                    # header 
                    header = ['frame'] + [f'{coord}_{i}' for i in range(21) for coord in ('x', 'y', 'z')]
                    writer.writerow(header)

                    # write data
                    for i, frame_data in enumerate(landmarks_seq):
                        writer.writerow([i] + frame_data)
                        
                print(f"Saved {len(landmarks_seq)} frames to {output_file}")
        elif (key == 27) or (key == ord('q')):  # ESC key to exit
            break

    cap.release()
    cv2.destroyAllWindows()

record_hand_landmarks()

Recording started...
Recording stopped.


Error: iterable expected, not int